In [1]:
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

MYPATH = 'E:/OSIC-Pul/'

In [2]:
def laplace_log_likelihood(y_true, y_pred, sigma):
    sigma_clipped = np.maximum(sigma, 70)
    delta_clipped = np.minimum(np.abs(y_true - y_pred), 1000)
    score = - np.sqrt(2) * delta_clipped / sigma_clipped - np.log(np.sqrt(2) * sigma_clipped)
    return np.mean(score)

In [5]:
trainData = pd.read_csv(MYPATH + 'train.csv')

# Encode categorical data to numerical
encoder = {"Sex": {"Male": 0, "Female": 1},
           "SmokingStatus": {"Never smoked": 0, "Ex-smoker": 0.5, "Currently smokes": 1}}
trainData.replace(encoder, inplace=True)

In [6]:
# For each FVC measurement, add a column with the patient's baseline FVC
# and when the measurement was taken in relation to the baseline
trainData['Patient_Week'] = trainData['Patient'] + '_' + trainData['Weeks'].astype(str)
patients = trainData.groupby(['Patient'])

trainData = pd.DataFrame(columns=trainData.columns)
trainData['Baseline_FVC'] = []
trainData['Weeks_Since'] = []
for _, patient in patients:
    patient.insert(len(patient.columns), 'Baseline_FVC', patient['FVC'].iloc[0])
    patient.insert(len(patient.columns), 'Weeks_Since', patient['Weeks'] - patient['Weeks'].iloc[0])
    trainData = trainData.append(patient)

                        Patient Weeks   FVC    Percent Age Sex  SmokingStatus  \
0     ID00007637202177411956430    -4  2315  58.253649  79   0            0.5   
1     ID00007637202177411956430     5  2214  55.712129  79   0            0.5   
2     ID00007637202177411956430     7  2061  51.862104  79   0            0.5   
3     ID00007637202177411956430     9  2144  53.950679  79   0            0.5   
4     ID00007637202177411956430    11  2069  52.063412  79   0            0.5   
...                         ...   ...   ...        ...  ..  ..            ...   
1544  ID00426637202313170790466    13  2712  66.594637  73   0            0.0   
1545  ID00426637202313170790466    19  2978  73.126412  73   0            0.0   
1546  ID00426637202313170790466    31  2908  71.407524  73   0            0.0   
1547  ID00426637202313170790466    43  2975  73.052745  73   0            0.0   
1548  ID00426637202313170790466    59  2774  68.117081  73   0            0.0   

                      Patie

In [ ]:
# Use age, sex, smoking status and baseline FVC to predict FVC at a specific week
X = trainData[['Age', 'Sex', 'SmokingStatus', 'Baseline_FVC', 'Weeks_Since']].to_numpy()
Y = trainData[['FVC']].to_numpy().ravel()

regr = LinearRegression()
x_train = X
y_train = Y
regr.fit(x_train, y_train)

# Predict FVC
y_pred = regr.predict(x_train)

# Predict Sigma using optimal sigma=sqrt(2)*delta
deltas = math.sqrt(2)*abs(y_train - y_pred)

# Values are exponentially distributed
lambda_hat = np.mean(deltas)
sigma_train = deltas
regs = LinearRegression()
regs.fit(x_train, sigma_train)
sigma_pred = regs.predict(x_train)

# Evaluate performance
print('FVC MSE: %.2f'% mean_squared_error(y_train, y_pred))
print('FVC R2: %.2f'% r2_score(y_train, y_pred))
print('Sigma Coefs: \n', regs.coef_)
print('Sigma MSE: %.2f'% mean_squared_error(sigma_train, sigma_pred))
print('Sigma R2: %.2f'% r2_score(sigma_train, sigma_pred))

stdev = np.std(deltas)
metric = []
for i in range(len(y_pred)):
    metric.append(laplace_log_likelihood(y_pred[i], y_train[i], sigma_pred[i]))

print('toy metric: ' + np.mean(metric).astype(str))

# Construct test input for Kaggle
test = pd.read_csv(MYPATH + 'test.csv').rename(columns={
        'FVC' : 'Baseline_FVC', 'Weeks' : 'Baseline_Weeks'})
test.replace(encoder, inplace=True)

submission = pd.read_csv(MYPATH + 'sample_submission.csv')
submission['Patient'] = submission['Patient_Week'].apply(lambda x: x.split('_')[0])
submission['Weeks'] = submission['Patient_Week'].apply(lambda x: x.split('_')[1]).astype(int)

test = submission.merge(test, on='Patient')
test['Weeks_Since'] = test['Weeks'] - test['Baseline_Weeks']

# Perform prediction
x_test = test[['Age', 'Sex', 'SmokingStatus', 'Baseline_FVC', 'Weeks_Since']].to_numpy()
y_test = regr.predict(x_test)
# Get confidence values
sigma_test = regs.predict(x_test)

test['FVC'] = y_test
test['Confidence'] = sigma_test

submission = test[['Patient_Week', 'FVC', 'Confidence']]

submission.to_csv('submission.csv', index=False)
